Скачаем репозиторий с гитхаб:

In [1]:
!git clone https://github.com/lcl-hse/LangExBank.git

fatal: destination path 'LangExBank' already exists and is not an empty directory.


Установим зависимости:

In [3]:
import os
os.chdir("LangExBank/LangExBank")

In [3]:
!pip install -r requirements.txt

     |████████████████████████████████| 7.9 MB 5.0 MB/s 
     |████████████████████████████████| 224 kB 48.8 MB/s 
     |████████████████████████████████| 45 kB 3.0 MB/s 
     |████████████████████████████████| 776.8 MB 18 kB/s 
     |████████████████████████████████| 64 kB 2.6 MB/s 
     |████████████████████████████████| 769 kB 62.7 MB/s 
     |████████████████████████████████| 2.7 MB 35.4 MB/s 
     |████████████████████████████████| 79 kB 7.6 MB/s 
     |████████████████████████████████| 1.2 MB 41.8 MB/s 
     |████████████████████████████████| 2.0 MB 34.9 MB/s 
  Created wheel for django-searchable-encrypted-fields: filename=django_searchable_encrypted_fields-0.1.9-py3-none-any.whl size=9806 sha256=791af26d5bb9203947e5e6e09548b717f746becea1467d30266de67c3940ff8a
  Stored in directory: /root/.cache/pip/wheels/b0/9d/c5/02d4ca400f61cd02a75f31f840733806c455f4cc9bc982a05c
Successfully built django-searchable-encrypted-fields
  Attempting uninstall: torch
    Found existing installation

Импортируем нужные модули и функции:

In [20]:
from testmaker import testmaker

import pandas as pd
import json

## Чтобы работали модели дистракторов:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

Выдачу вопросов в testmaker можно получить одной строкой кода - testmaker сам всё скачает, распарсит и удалит после завершения (может занимать много времени):

In [21]:
help(testmaker.download_folder_and_make_exercises)

Help on function download_folder_and_make_exercises in module testmaker.testmaker:

download_folder_and_make_exercises(folder_name, output_path=None, maintain_log=False, error_types=[], context=True, make_two_variants=True, file_output=True, moodle_output=True, check_duplicates=True, keep_processed=False, path_to_downloaded='downloaded_2022_04_08_22_53_34270461', delete_downloaded=False, filter_query=None, keep_all_exercises=False, keep_intersec_tags=False)



Немного об аргументах функции:

folder_name - Какую папку из REALEC мы хотим скачать

output_path - В какую папку сохранить выдачу

maintain_log - Сохранять ли лог генерации вопросов

error_types - Какие типы ошибок нас интересуют

context - Включать ли в вопросы контекст из предложения до и предложения после

make_two_variants - Делать ли из каждого предложения два упражнения - два варианта (по умолчанию: одно предложение - одно упражнение). Работает только в testmaker_old

file_output - Сохранять ли выдачу в файл (Если поставить False, то приведёт к ошибке при make_two_variants==True) Сохранить можно только в формате Moodle

moodle_output - Сохранять ли выдачу в формате Moodle

check_duplicates - Проверять ли эссе в REALEC на дупликаты

keep_processed - Сохранять ли предобработанные тексты на жёстком диске (может быть полезно для дебага)

path_to_downloaded - Куда сохранять скачанные из REALEC файлы

delete_downloaded - Удалять ли скачанные из REALEC файлы после завершения работы функции

Попробуем получить выдачу на тэги ошибки Articles и Choice of tense из папки Exam2016:

In [22]:
output1 = testmaker.download_folder_and_make_exercises(folder_name='/exam/Exam2016/',
                                            error_types=['Articles','Tense_choice'],
                                            file_output=False, moodle_output=False,
                                            keep_processed=False, make_two_variants=False,
                                            context=True, delete_downloaded=True)

downloading collection /exam/Exam2016/ ...
downloaded, extracting... 
folder Exam2016 from realec.org succesfully saved to downloaded_2022_04_08_22_53_34270461
Filter query - None
collecting errors info...
Making exercises...
Writing short_answer questions, 1463 total ...
done, saved in RAM as BytesIO object


А что у нас получилось на выходе функции?

In [23]:
type(output1)

dict

In [24]:
[key for key in output1]

['short_answer']

Словарь тип вопроса - вопросы

In [25]:
type(output1['short_answer'])

list

In [26]:
output1['short_answer'][:10]

[(" It is getting harder and harder for families to pay for everything starting from rent, medicine, food and ending with children's education and leisure time. Trying to cope with family budget problems, women often <b>had</b> to work. Trying to give children more, they take away from them the most important thing - communication with their parents.",
  ['have'],
  False,
  'Tense_choice',
  {'text_type': None,
   'ielts': 'TRUE ',
   'filename': '2016_AKo_2_2',
   'folder': 'downloaded_2022_04_08_22_53_34270461\\Exam2016'}),
 ('The demonstrated bar charts provide us with information on how the use of Facebook changed from 2012 to 2013 in millions of users and the percentages of men and women indicating one of the given reasons for using Facebook as main. From the first diagram we can see that the number of desktop Facebook users slightly decreased in 2013 (about 130 million) in comparison to 2012 (about 150 million), however, the number of users of the mobile version <b>grew dramatic

Элементы каждого кортежа в этом списке:

- Предложение, в котором область ошибки выделена <b>жирным</b> шрифтом с помощью тэга \<b\>\</b\>
- Правильный ответ
- Правда ли то, что этот вопрос - единственный, полученный из соответствующего предложения (релевантно при генерации нескольких вариантов)
- Тип ошибки
- Словарь
  - Filename - Имя файла, из которого предложение
  - Folder - Имя папки, из которой файл
  - ielts - Был ли файл, из которого взяли предложение айлтсовским сочинением
  - text_type - Тип текста, из которого взяли предложение

Попробуем получить такие же предложения без контекста, и сохранить выдачу (в таком случае функция вернёт пути к файлам)

In [27]:
output2 = testmaker.download_folder_and_make_exercises(folder_name='/exam/Exam2016/', error_types=['Articles','Tense_choice'],
                                            file_output=True, moodle_output=True,
                                            keep_processed=False, make_two_variants=False,
                                            context=False, delete_downloaded=True)

downloading collection /exam/Exam2016/ ...
downloaded, extracting... 
folder Exam2016 from realec.org succesfully saved to downloaded_2022_04_08_22_53_34270461
C:\Users\User\HSE\LangExBank\LangExBank\ moodle_exercises
Filter query - None
collecting errors info...
Making exercises...
Writing short_answer questions, 2072 total ...
done, saved to: C:\Users\User\HSE\LangExBank\LangExBank\moodle_exercises


moodle_exercises - название папки с выдачей по умолчанию

In [28]:
!ls moodle_exercises

"ls" ­Ґ пў«пҐвбп ў­гваҐ­­Ґ© Ё«Ё ў­Ґи­Ґ©
Є®¬ ­¤®©, ЁбЇ®«­пҐ¬®© Їа®Ја ¬¬®© Ё«Ё Ї ЄҐв­л¬ д ©«®¬.


Также можно выгрузить только предложения, соответствующие определённому текстовому запросу (поддерживаются регулярные выражения):

In [30]:
output3 = testmaker.download_folder_and_make_exercises(folder_name='/exam/',
                                                       error_types=['Tense_choice','lex_item_choice','Prepositions'],
                                                       filter_query='students',
                                                       file_output=False, moodle_output=False,
                                                       keep_processed=False, make_two_variants=False,
                                                       context=False, delete_downloaded=True)

downloading collection /exam/ ...
downloaded, extracting... 
folder exam from realec.org succesfully saved to downloaded_2022_04_08_22_53_34270461
Filter query - students
collecting errors info...
Making exercises...
Writing short_answer questions, 377 total ...
done, saved in RAM as BytesIO object


In [31]:
data = [(i[0], i[1][0], i[3],
         i[4]['folder'], i[4]['filename']) for i in output3['short_answer']]
data = pd.DataFrame(data, columns=['Sentence', 'Right answer', 'Error type',
                                     'Folder', 'Filename'])

In [32]:
data

,Sentence,Right answer,Error type,Folder,Filename
0,"For instance, students are overloaded <b>by</...",with,Prepositions,downloaded_2022_04_08_22_53_34270461\exam\best...,39_2
1,This fact will increase the <b>effectivity</b...,efficiency,lex_item_choice,downloaded_2022_04_08_22_53_34270461\exam\Exam...,2014_KKo_15_2
2,Some students in high schools just can't spen...,youngsters,lex_item_choice,downloaded_2022_04_08_22_53_34270461\exam\Exam...,2014_KKo_23_2
3,I believe that charity <b>of</b> different le...,on,Prepositions,downloaded_2022_04_08_22_53_34270461\exam\Exam...,2014_KKo_25_2
4,"In addition, some students are afraid of resp...",with,Prepositions,downloaded_2022_04_08_22_53_34270461\exam\Exam...,2014_KKo_25_2
...,...,...,...,...,...
372,"Moreover, science discoveries, which were mad...",in,Prepositions,downloaded_2022_04_08_22_53_34270461\exam\Exam...,2014_EEm_39_2
373,"According to the given theory, 2% or 3% of fe...",becoming,lex_item_choice,downloaded_2022_04_08_22_53_34270461\exam\Exam...,2014_EEm_9_2
374,"Also, all male students have to be accepted, ...",regardless,lex_item_choice,downloaded_2022_04_08_22_53_34270461\exam\Exam...,2014_EEm_9_2
375,The highest proportion of <b>men</b> got skil...,male students,lex_item_choice,downloaded_2022_04_08_22_53_34270461\exam\Exam...,2014_ESha_12_1


Отлично, теперь попробуем сделать из этой выдачи вопросы multiple_choice при помощи distractor_generator. Типы ошибок, которые поддерживаются моделями генерации дистракторов - Choice of a lexical item(lex_item_choice), Choice of tense (Tense_choice) и Prepositions

In [33]:
from DisGen.distractor_generator import get_distractors

In [35]:
!python -m spacy download en_core_web_sm

     ---------------------------------------- 13.9/13.9 MB 5.4 MB/s eta 0:00:00
[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [36]:
%%time
data = get_distractors(data)

C:\Users\User\HSE\LangExBank\LangExBank\DisGen\distractor_generator\distractor_models.py:78: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Contains verb'] = df['Right answer'].apply(lambda x: contains_pos(x, ('VERB','AUX'),self.parser))
C:\Users\User\HSE\LangExBank\LangExBank\DisGen\distractor_generator\distractor_models.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Sentence'] = df['Sentence'].apply(lambda x: re.sub('<b>.*?</b>','_'*8, x))
C:\Users\User\HSE\LangExBank\LangExBank\DisGen\distra

C:\Users\User\HSE\LangExBank\LangExBank\DisGen\distractor_generator\nn_models.py:114: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  pred = F.softmax(model(batch_device).data.cpu())
C:\Users\User\HSE\LangExBank\LangExBank\DisGen\distractor_generator\distractor_models.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Sentence'] = df['Sentence'].apply(lambda x: re.sub('<b>.*?</b>','_'*8, x))
C:\Users\User\HSE\LangExBank\LangExBank\DisGen\distractor_generator\distractor_models.py:187: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation:

CPU times: total: 16.9 s
Wall time: 13.2 s


C:\Users\User\HSE\LangExBank\LangExBank\DisGen\distractor_generator\distractor_models.py:195: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[['Keep 2','Wrong answer lemma', 'POS Wrong', 'Tag Wrong']] = df.apply(filter_func_w, axis=1,
C:\Users\User\HSE\LangExBank\LangExBank\DisGen\distractor_generator\distractor_models.py:195: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[['Keep 2','Wrong answer lemma', 'POS Wrong', 'Tag Wrong']] = df.apply(filter_func_w, axis=1,
C:\Users\User\HSE\LangExBank\LangExBank\

Посмотрим, что у нас получилось на выходе:

In [37]:
type(data), len(data)

(pandas.core.frame.DataFrame, 259)

In [38]:
data.head()

,Sentence,Right answer,Wrong answer,Distractor 1,Distractor 2,Distractor 3,Error type,Folder,Filename
0,"For instance, students are overloaded _______...",with,by,in,on,and,Prepositions,downloaded_2022_04_08_22_53_34270461\exam\best...,39_2
1,This fact will increase the ________ of the v...,efficiency,effectivity,level,development,number,lex_item_choice,downloaded_2022_04_08_22_53_34270461\exam\Exam...,2014_KKo_15_2
2,Some students in high schools just can't spen...,youngsters,youngers,chances,appearances,states,lex_item_choice,downloaded_2022_04_08_22_53_34270461\exam\Exam...,2014_KKo_23_2
3,I believe that charity ________ different lev...,on,of,with,for,through,Prepositions,downloaded_2022_04_08_22_53_34270461\exam\Exam...,2014_KKo_25_2
4,"In addition, some students are afraid of resp...",with,in,by,on,over,Prepositions,downloaded_2022_04_08_22_53_34270461\exam\Exam...,2014_KKo_25_2


In [39]:
data['Sentence'].to_list()[:10]

[' For instance, students are overloaded ________ their schoolwork and extracurricular activities, which lresults into less time for hobbies and also for the communication with relatives.',
 ' This fact will increase the ________ of the volunteer work and also will increase the number of students, involved in this work.',
 " Some students in high schools just can't spend extra time helping the ________ because of hard exam preparation.",
 ' I believe that charity ________ different levels improves our world so students which help homeless people or plant trees in a park make our country better.',
 ' In addition, some students are afraid of responsibility and have problems ________ talking in front of people.',
 " It isn't easy to add community service ________ high school program, but some years later students will understand the usefulness of it.",
 ' In this situation, if universities should accept equal ________ of male and female students, it will be impossible for some boys to stu